### IMPORTANT NOTE:
Streamlit before 0.57 only works with Bokeh 1.0 and Streamlit 0.57+ only works with Bokeh 2.<br>
I have used **Streamlit 0.62.0** and **Bokeh 2.1.1**

In [11]:
# Install Streamlit and Bokeh
!pip install streamlit
!pip install bokeh

In [77]:
# Check which version is in your computer
!streamlit --version
!bokeh info

Streamlit, version 0.62.0
Python version      :  3.7.4 (default, Aug 13 2019, 20:35:49) 
IPython version     :  7.8.0
Tornado version     :  6.0.3
Bokeh version       :  2.1.1
BokehJS static path :  /home/angelrps/anaconda3/lib/python3.7/site-packages/bokeh/server/static
node.js version     :  (not installed)
npm version         :  (not installed)


In [ ]:
!streamlit hello

In [7]:
%%writefile streamlit_app.py

import streamlit as st
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, MultiPolygon
import matplotlib.pyplot as plt
import random

st.title("Taxi Demand Predict")
"""
Write some description here
"""

@st.cache
def load_shape_data():
    df_taxizones = gpd.read_file('../data/taxi_zones/taxi_zones.shp')

    # filter Manhattan zones
    df_taxizones = df_taxizones[df_taxizones['borough'] == 'Manhattan'].reset_index(drop=True)

    df_taxizones = df_taxizones.drop(['borough'], axis=1)

    df_taxizones.to_crs(epsg=3785, inplace=True)

    #EPSG-Code of Web Mercador

    # Simplify Shape of Zones (otherwise slow peformance of plot)
    df_taxizones["geometry"] = df_taxizones["geometry"].simplify(100)

    data = []
    for zonename, LocationID, shape in df_taxizones[["zone", "LocationID", "geometry"]].values:
        #If shape is polygon, extract X and Y coordinates of boundary line:
        if isinstance(shape, Polygon):
            X, Y = shape.boundary.xy
            X = [int(x) for x in X]
            Y = [int(y) for y in Y]
            data.append([LocationID, zonename, X, Y])

        #If shape is Multipolygon, extract X and Y coordinates of each sub-Polygon:
        if isinstance(shape, MultiPolygon):
            for poly in shape:
                X, Y = poly.boundary.xy
                X = [int(x) for x in X]
                Y = [int(y) for y in Y]
                data.append([LocationID, zonename, X, Y])

    #Create new DataFrame with X an Y coordinates separated:
    df_taxizones = pd.DataFrame(data, columns=["LocationID", "ZoneName", "X", "Y"])
    return df_taxizones

df_taxizones1 = load_shape_data()

@st.cache(allow_output_mutation=True)
def load_taxis_data(df_taxizones1):
    # 'df_taxis' should be a data frame returned by the ML model
    df_taxis = pd.read_csv('../data/Data_Taxis_2017_Cleaned.csv', sep=',',
                          usecols=['hour','dayofweek','LocationID','NoOfPickups'])

    pickups = df_taxis.groupby(['hour','dayofweek','LocationID']).sum()

    for hour in range(24):    
        for dayofweek in range(7):

            #Get pickups and dropoff for this hour and weekday:
            p = pd.DataFrame(pickups.loc[(hour, dayofweek)]).reset_index().rename(columns={"LocationID" : "LocationID"})

            #Add information of pickups and dropoff to the New York Taxi Zone DataFrame:
            df_taxizones1 = pd.merge(df_taxizones1, p, on="LocationID", how="left").fillna(0)
            df_taxizones1.rename(columns={"NoOfPickups" : "Passenger_%d_%d"%(dayofweek, hour)}, inplace=True) 
    return df_taxizones1

df_taxizones2 = load_taxis_data(df_taxizones1)

from bokeh.io import output_notebook, output_file, show
from bokeh.plotting import figure
from bokeh.models import HoverTool, Select, ColumnDataSource, WheelZoomTool, LogColorMapper, LinearColorMapper, ColorBar, BasicTicker
from bokeh.palettes import Viridis256 as palette
from bokeh.layouts import row

# add slider widget: Hours
hour = st.slider("Hour",min_value=0, max_value=23, value=7, step=1)

# add slider widget: Dayofweek
weekday = st.slider("Day of week",min_value=0, max_value=6, value=1, step=1)

# ColumnDataSource transforms the data into something that Bokeh and Java understand
df_taxizones2["Passengers"] = df_taxizones2["Passenger_" + str(weekday) + "_" + str(hour)]

source = ColumnDataSource(df_taxizones2)

max_passengers_per_hour = df_taxizones2[filter(lambda x: "Passenger_" in x, df_taxizones2.columns)].max().max()

color_mapper = LinearColorMapper(palette=palette[::-1], high=max_passengers_per_hour, low=0)

##### Color Bar
color_bar = ColorBar(color_mapper = color_mapper,
                     ticker = BasicTicker(),
                    label_standoff=8,
                     location=(0,0),
                     orientation='vertical')

p = figure(title="New York Taxi Pickups",
           plot_width=450, plot_height=750,
           toolbar_location=None,
           tools='pan,wheel_zoom,box_zoom,reset,save')
p.xaxis.visible = False
p.yaxis.visible = False

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

#Get rid of zoom on axes:
for t in p.tools:
    if type(t) == WheelZoomTool:
        t.zoom_on_axis = False

patches = p.patches(xs="X", ys="Y", source=source,fill_alpha=1,
                  fill_color={'field': 'Passengers',
                              'transform': color_mapper},
                  line_color="black", alpha=0.5)

hovertool = HoverTool(tooltips=[('Zone:', "@ZoneName"),
                                ("Passengers:", "@Passengers")])
p.add_tools(hovertool)

p.add_layout(color_bar, 'right')

st.bokeh_chart(p)
max_value = df_taxizones2.drop(['LocationID','ZoneName','X', 'Y'], axis=1).max().sort_values()

st.write(max_value)
"""
End of script
"""

Overwriting streamlit_app.py


### Writing function to create input data for the predictive model
I have to create a table with the following columns:

```- month```<br>
```- day```<br>
```- hour```<br>
```- week```<br>
```- dayofweek```<br>
```- isweekend```<br>
```- precipitation```

In [22]:
import numpy as np
import pandas as pd
from datetime import date, timedelta
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Get start date
today = date.today()
start_date = today + timedelta(days=1)

NoOfDays = 3 # define number of days for prediction

# create data frame with range of days by hour
df_pred = pd.DataFrame()
dates = pd.date_range(start = start_date, end = start_date + timedelta(days=NoOfDays), freq = "H")
df_pred['datetime'] = dates
df_pred.drop([df.shape[0]], inplace=True)

# Create new columns from datetime
df_pred['year'] = df_pred['datetime'].dt.year
df_pred['month'] = df_pred['datetime'].dt.month
df_pred['day'] = df_pred['datetime'].dt.day
df_pred['hour'] = df_pred['datetime'].dt.hour
df_pred['week'] = df_pred['datetime'].dt.week
df_pred['dayofweek'] = df_pred['datetime'].dt.dayofweek
    # create column 'isweekend'
mask = (df_pred['dayofweek'] == 5) | (df_pred['dayofweek'] == 6)
df_pred['isweekend'] = np.where(mask, 1, 0)

df_pred

,datetime,year,month,day,hour,week,dayofweek,isweekend
0,2020-07-25 00:00:00,2020,7,25,0,30,5,1
1,2020-07-25 01:00:00,2020,7,25,1,30,5,1
2,2020-07-25 02:00:00,2020,7,25,2,30,5,1
3,2020-07-25 03:00:00,2020,7,25,3,30,5,1
4,2020-07-25 04:00:00,2020,7,25,4,30,5,1
5,2020-07-25 05:00:00,2020,7,25,5,30,5,1
6,2020-07-25 06:00:00,2020,7,25,6,30,5,1
7,2020-07-25 07:00:00,2020,7,25,7,30,5,1
8,2020-07-25 08:00:00,2020,7,25,8,30,5,1
9,2020-07-25 09:00:00,2020,7,25,9,30,5,1


In [ ]:
# Create new columns from datetime
df_pred['year'] = df_pred['datetime'].dt.year
df_pred['month'] = df_pred['datetime'].dt.month
df_pred['day'] = df_pred['datetime'].dt.day
df_pred['hour'] = df_pred['datetime'].dt.hour
df_pred['week'] = df_pred['datetime'].dt.week
df_pred['dayofweek'] = df_pred['datetime'].dt.dayofweek
    # create column 'isweekend'
mask = (df_pred['dayofweek'] == 5) | (df_pred['dayofweek'] == 6)
df_pred['isweekend'] = np.where(mask, 1, 0)